In [1]:
import onnx
import numpy as np
import pandas as pd
from onnx import numpy_helper
import os

# Load the ONNX model
model_path = "/kaggle/input/vggweightstointarsh/tensorflow2/default/1/vgg16-7.onnx"
onnx_model = onnx.load(model_path)

# Output CSV path
output_csv = "/kaggle/working/VGG_Float_Layerwise.csv"

# Remove file if it already exists
if os.path.exists(output_csv):
    os.remove(output_csv)

# Flag to know if it's the first column (to write header)
first_column = True

# Stream weights layer by layer
for initializer in onnx_model.graph.initializer:
    layer_name = initializer.name

    # Skip non-weight data if needed (optional)
    if not (".weight" in layer_name or "conv" in layer_name or "fc" in layer_name):
        continue

    # Flatten and reshape to a column vector
    weights = numpy_helper.to_array(initializer).flatten()
    df_col = pd.DataFrame({layer_name: weights})

    # Append column to CSV
    if first_column:
        df_col.to_csv(output_csv, index=False)
        first_column = False
    else:
        # Read existing CSV and merge column-wise
        existing_df = pd.read_csv(output_csv)
        merged_df = pd.concat([existing_df, df_col], axis=1)
        merged_df.to_csv(output_csv, index=False)

    print(f"✅ Processed: {layer_name} with {len(weights)} weights")

print("\n✅ All layers written to:", output_csv)



✅ Processed: vgg0_conv0_weight with 1728 weights
✅ Processed: vgg0_conv0_bias with 64 weights
✅ Processed: vgg0_conv1_weight with 36864 weights
✅ Processed: vgg0_conv1_bias with 64 weights
✅ Processed: vgg0_conv2_weight with 73728 weights
✅ Processed: vgg0_conv2_bias with 128 weights
✅ Processed: vgg0_conv3_weight with 147456 weights
✅ Processed: vgg0_conv3_bias with 128 weights
✅ Processed: vgg0_conv4_weight with 294912 weights
✅ Processed: vgg0_conv4_bias with 256 weights
✅ Processed: vgg0_conv5_weight with 589824 weights
✅ Processed: vgg0_conv5_bias with 256 weights
✅ Processed: vgg0_conv6_weight with 589824 weights
✅ Processed: vgg0_conv6_bias with 256 weights
✅ Processed: vgg0_conv7_weight with 1179648 weights
✅ Processed: vgg0_conv7_bias with 512 weights
✅ Processed: vgg0_conv8_weight with 2359296 weights
✅ Processed: vgg0_conv8_bias with 512 weights
✅ Processed: vgg0_conv9_weight with 2359296 weights
✅ Processed: vgg0_conv9_bias with 512 weights
✅ Processed: vgg0_conv10_weight w

In [4]:
import pandas as pd
import numpy as np

# Load the CSV file (adjust path as needed)
csv_path = '/kaggle/working/VGG_Float_Layerwise.csv'
df = pd.read_csv(csv_path)

# Check for any non-numeric or NaN values
df = df.apply(pd.to_numeric, errors='coerce')  # Coerce non-numeric values to NaN
df = df.fillna(0)  # Replace NaNs with 0 (or use another strategy if needed)

# Get global min and max
min_val = df.min().min()
max_val = df.max().max()

# Handle case where min == max to avoid division by zero
if min_val == max_val:
    scale = 1
else:
    scale = 255 / (max_val - min_val)
# Vectorized conversion function
def float_to_int8_vectorized(x):
    return np.round((x - min_val) * scale - 128).astype(np.int8)

# Apply the function using vectorized approach
df_int8 = df.apply(float_to_int8_vectorized)

# Save the result
output_path = '/kaggle/working/VGG_Int8_Weights.csv'
df_int8.to_csv(output_path, index=False)

print(f"✅ Conversion completed and saved to {output_path}")
    

✅ Conversion completed and saved to /kaggle/working/VGG_Int8_Weights.csv
